# *Using APIs*

## 1. Parsing JSON

Using *freegeoip.net* we are able to resolves IP addresses to physical addresses. I can take the output of this request and use Python's JSon parsing functions to decode it.

In [ ]:
import json
from urllib.request import urlopen

def getCountry(ipAddress):
    response = urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("50.78.253.58"))

## 2. Create a Basic script that crawls Wikipedia

This codes will allow us to looks for revision history pages and then looks for IP addresses.

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    #Extract all internal links in a page related to a specific articleUrl
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html)
    return bsObj.find("div",{"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))

def getHistoryIPs(pageUrl):
    pageUrl = pageUrl.replace("/wiki/","")
    historyUrl = "http://en.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"
    print("history url is: "+historyUrl)
    html = urlopen(historyUrl)
    bsObj = BeautifulSoup(html)
    ipAddresses = bsObj.findAll("a",{"class":"mw-anounserlink"})
    addressList = set()
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
    return addressList

links = getLinks("/wiki/Python_(programming_language)")

while(len(links) > 0):
    for link in links:
        print('----------------')
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            print(historyIP)
        
    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)

----------------
history url is: http://en.wikipedia.org/w/index.php?title=Programming_paradigm&action=history
----------------
history url is: http://en.wikipedia.org/w/index.php?title=Multi-paradigm_programming_language&action=history
----------------
history url is: http://en.wikipedia.org/w/index.php?title=Object-oriented_programming&action=history
----------------
history url is: http://en.wikipedia.org/w/index.php?title=Procedural_programming&action=history
----------------
history url is: http://en.wikipedia.org/w/index.php?title=Imperative_programming&action=history
----------------
history url is: http://en.wikipedia.org/w/index.php?title=Functional_programming&action=history


KeyboardInterrupt: 